In [ ]:
import random
import sys
import scipy
import pickle 
import argparse
import warnings
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm
import torch.distributions as dist

warnings.filterwarnings('ignore')

##Data Load

In [ ]:
y = np.load('/content/drive/MyDrive/Colab Notebooks/Deep_Compact_Perception/MUStARD/sarcasm_labels.npy')
x_v = np.load('/content/drive/MyDrive/Colab Notebooks/Deep_Compact_Perception/MUStARD/video_features.npy')
print(x_v.shape)
x_a = np.load('/content/drive/MyDrive/Colab Notebooks/Deep_Compact_Perception/MUStARD/audio_fea.npy')[:,1:282]
print(x_a.shape)
x_t = np.load('/content/drive/MyDrive/Colab Notebooks/Deep_Compact_Perception/MUStARD/text_bert_embeddings.npy')
print(x_t.shape)

(690, 2048)
(690, 281)
(690, 768)


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_t_normalized = scaler.fit_transform(x_t)
x_a_normalized = scaler.fit_transform(x_a)
x_v_normalized = scaler.fit_transform(x_v)

## NN Model

In [ ]:
############################### With Linear ##################################
class IBH(nn.Module):
    def __init__(self, input_dims, para_shift):
        super(IBH, self).__init__()

        self.X0_dim = input_dims['X0_dim']
        self.X1_dim = input_dims['X1_dim']
        self.X2_dim = input_dims['X2_dim']
        self.inter_dim1 = input_dims['inter_dim1']
        self.inter_dim2 = input_dims['inter_dim2']
        self.drop_prob = input_dims['drop_prob']
        self.B0_dim = input_dims['B0_dim']
        self.B1_dim = input_dims['B1_dim']
        
        self.p_beta = para_shift['p_beta']
        self.p_gamma = para_shift['p_gamma']
        self.p_lambda = para_shift['p_lambda']
        
        self.encoder1 = nn.Sequential(
            nn.Linear(self.X0_dim, self.inter_dim1),
            nn.ReLU(),
            nn.Dropout(self.drop_prob),
            nn.Linear(self.inter_dim1, self.B0_dim * 2),
            # Dont add ReLU anymore!
        )
        
        self.decoder1 = nn.Sequential(
            nn.Linear(self.B0_dim, self.inter_dim1),
            nn.ReLU(),
            nn.Dropout(self.drop_prob),
            nn.Linear(self.inter_dim1, self.X1_dim),
            nn.Sigmoid(),
            nn.Dropout(self.drop_prob),
        )       

        self.encoder2 = nn.Sequential(
            nn.Linear(self.B0_dim, self.inter_dim1),
            nn.ReLU(),
            nn.Dropout(self.drop_prob),
            nn.Linear(self.inter_dim1, self.B1_dim * 2),
            # Dont add ReLU anymore!
        )

        self.decoder2 = nn.Sequential(
            nn.Linear(self.B1_dim, self.inter_dim1),
            nn.ReLU(),
            nn.Dropout(self.drop_prob),
            nn.Linear(self.inter_dim1, self.X2_dim),
            nn.Sigmoid(),
            nn.Dropout(self.drop_prob),
        )

        self.fc1 = nn.Linear(self.B1_dim, self.inter_dim2)
        self.fc2 = nn.Linear(self.inter_dim2, 1)

        self.criterion = nn.MSELoss()

    def kl_loss(self, mu, logvar):
        kl_div = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp(), dim=-1)
        kl_mean = torch.mean(kl_div)
        return kl_mean

    def reparameterise(self, mu, logvar):
        epsilon = torch.randn_like(mu)
        return mu + epsilon * torch.exp(logvar / 2)

    def forward(self, x0, x1, x2):
        h1 = self.encoder1(x0)
        mu1, logvar1 = h1.chunk(2, dim=-1)
        kl_loss_0 = self.kl_loss(mu1, logvar1)
        b0 = self.reparameterise(mu1, logvar1)
        output1 = self.decoder1(b0)

        mse_0 = self.criterion(output1, x1)
        IB0 = kl_loss_0 + self.p_beta * mse_0
        
        b0_sample = b0
        h2 = self.encoder2(b0_sample)
        mu2, logvar2 = h2.chunk(2, dim=-1)

        kl_loss_1 = self.kl_loss(mu2, logvar2)
        b1 = self.reparameterise(mu2, logvar2)
        output2 = self.decoder2(b1)
        mse_1 = self.criterion(output2, x2)
        IB1 = kl_loss_1 + self.p_gamma * mse_1
        IB_total = IB0 + self.p_lambda * IB1
        
        b1 = b1.view(b1.size(0), -1)
        b1 = torch.relu(self.fc1(b1))
        y_pred = torch.sigmoid(self.fc2(b1))

        return y_pred.squeeze(), IB_total

In [ ]:
############################### With Linear ##################################
class Unimodel(nn.Module):
    def __init__(self, input_dim):
        super(Unimodel, self).__init__()

        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        y_pred = torch.sigmoid(self.fc2(x))

        return y_pred.squeeze()

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, recall_score
from sklearn.metrics import classification_report
n_splits = 5
kf = KFold(n_splits=n_splits, random_state=42,shuffle=True)

In [ ]:
pre_list = [[] for i in range(n_splits)]
recall_list = [[] for i in range(n_splits)]
f1_list = [[] for i in range(n_splits)]

input_dim = x_t.shape[-1]

for fold, (train_index, test_index) in enumerate(kf.split(x_t)):
    model = Unimodel(input_dim)
    loss_func = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    print(f"Fold {fold + 1}")

    x_train, x_test = x_t[train_index], x_t[test_index]
    y_train_np, y_test_np = y[train_index], y[test_index]

    x_t_train = torch.tensor(x_train, dtype=torch.float32)
    x_t_test = torch.tensor(x_test, dtype=torch.float32)

    y_train = torch.tensor(y_train_np,dtype=torch.float32)
    y_test = torch.tensor(y_test_np,dtype=torch.float32)

    # Training
    for epoch in range(200):
        model.train()
        
        # Forward pass
        y_pred = model(x_t_train)
        
        # Compute loss
        loss = loss_func(y_pred, y_train)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Evaluate on the test set
        model.eval()
        with torch.no_grad():
            y_pred_test = model(x_t_test)
            y_pred_test = (y_pred_test > 0.5).float()
            result = classification_report(y_test.cpu().numpy(), y_pred_test.cpu().numpy(), output_dict=True, digits=3)

            # Calculate F1-score and recall. Note that these functions expect numpy arrays.
            pre_list[fold].append(result["weighted avg"]["precision"])
            recall_list[fold].append(result["weighted avg"]["recall"])
            f1_list[fold].append(result["weighted avg"]["f1-score"])

Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
